In [1]:
import numpy as np
from os import listdir, mkdir, path
import matplotlib.pyplot as plt
from matplotlib import colors, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
from scipy.signal import medfilt
import re
from sklearn import tree
%matplotlib
#import pdb; pdb.set_trace()

Using matplotlib backend: TkAgg


In [2]:
folder1 = '/Users/Amanda/Desktop/PtSn/PtSn1/'
folder2 = '/Users/Amanda/Desktop/PtSn/PtSn2/'
bgfile1 = '/Users/Amanda/Desktop/PtSn/PtSn1/PtSn1-background.dat'
bgfile2 = '/Users/Amanda/Desktop/PtSn/PtSn2/PtSn2--background.dat'
loc1 = path.join(folder1, 'addtnl', 'PtSn1')
loc2 = path.join(folder2, 'addtnl', 'PtSn2')

sequencestarters1 = ['/Users/Amanda/Desktop/PtSn/PtSn1/PtSn1_scan1_0000_C.dat',
                    '/Users/Amanda/Desktop/PtSn/PtSn1/PtSn1_1_scan1_0000_C.dat',
                    '/Users/Amanda/Desktop/PtSn/PtSn1/PtSn1_2_scan1_0000_C.dat']
sequencestarters2 = ['/Users/Amanda/Desktop/PtSn/PtSn2/_PtSn2_scan1_0000_C.dat',
                     '/Users/Amanda/Desktop/PtSn/PtSn2/PtSn2_1_scan1_0000_C.dat',
                     '/Users/Amanda/Desktop/PtSn/PtSn2/PtSn2_2_scan1_0000_C.dat']

In [ ]:
# I1,q1,sigma1,I2,sigma2
fcc_bcc_bounds1_I = ([0,2.69,0.025, 0,0.025, 0,2.80,0.025, 0,0.025],
                     [4,2.79,0.08,  4,0.08,  4,2.9,0.08,   4,0.08])
fcc_bcc_initialvalues1_I = [[1.29, 2.74,0.05, 0.225,0.05,  0.125,2.85,0.05, 0.125,0.05],
                            [0.125,2.74,0.05, 0.125,0.05,  0.636,2.85,0.05, 0.300,0.05],
                            [0.087,2.74,0.05, 0.037,0.05,  0.708,2.85,0.05, 0.401,0.05]]
fcc_bcc_bounds2_I = ([0,2.69,0.025, 0,0.025, 0,2.80,0.025, 0,0.025],
                     [4,2.79,0.08,  4,0.08,  4,3.06,0.08,   4,0.08])
fcc_bcc_initialvalues2_I = [[1.027,2.74,0.05, 0.125,0.05, 0.125,3.01,0.05, 0.250,0.05],
                            [0.689,2.74,0.05, 0.062,0.05, 0.062,3.01,0.05, 0.137,0.05],
                            [0.714,2.74,0.05, 0.062,0.05, 0.062,3.01,0.05, 0.162,0.05]]

sigma2ratio measured on high-SNR plots represents a lower extremeness limit for the physical ratio, which can always be pushed towards 1:1 by lower instrumental resolution.  It seems unlikely for this low-SNR data to have better spatial resolution than the exemplar hiSNR spectra.

Integrated intensity is a truly physical parameter, not dependent on instrumentation. Not sure how much it should vary with composition; tried to set reasonable boundaries.

In [3]:
# I1,q1,sigma1,I2ratio,sigma2ratio
bounds1 = ([0,2.69,0.025, 0.2,0.8, 0,2.80,0.025, 0,0.025],
           [4,2.79,0.08,  0.5,1.1, 4,2.9,0.08,   4,0.08])
initialvalues1 = [[1.29, 2.74,0.05, 0.2,0.94,  0.125,2.85,0.05, 0.125,0.05],
                  [0.125,2.74,0.05, 0.5,0.94,  0.636,2.85,0.05, 0.300,0.05],
                  [0.087,2.74,0.05, 0.5,0.94,  0.708,2.85,0.05, 0.401,0.05]]
# I1,q1,sigma1,I2,sigma2
bounds2 = ([0,2.69,0.025, 0.2,0.8, 0,2.80,0.025, 0,0.025],
           [4,2.79,0.08,  0.5,1.1, 4,3.06,0.08,  4,0.08])
initialvalues2 = [[1.027,2.74,0.05, 0.370,0.939, 0.125,3.01,0.05, 0.250,0.05],
                  [0.689,2.74,0.05, 0.350,0.937, 0.062,3.01,0.05, 0.137,0.05],
                  [0.714,2.74,0.05, 0.320,0.935, 0.062,3.01,0.05, 0.162,0.05]]

Functions for reading in files

In [4]:
def mk_subfolders(folder):
    if not path.isdir(path.join(folder, 'addtnl')):
        mkdir(path.join(folder, 'addtnl'))
    if not path.isdir(path.join(folder, 'pdf')):
        mkdir(path.join(folder, 'pdf'))
    if not path.isdir(path.join(folder, 'png')):
        mkdir(path.join(folder, 'png'))

def find_n_order_files_PtSn(folder):
    files = listdir(folder)
    files2 = [ii for ii in files if ((not ii.__contains__('background')) & (not ii.__contains__('seeds')) & (not ii.__contains__('pdf')))]
    seq1 = [ii for ii in files2 if re.match('[_]*PtSn[12]_scan1.*', ii)]
    seq2 = [ii for ii in files2 if re.match('PtSn[12]_1_scan1.*', ii)]
    seq3 = [ii for ii in files2 if re.match('PtSn[12]_2_scan1.*', ii)]
    seq = seq1
    seq.extend(seq2)
    seq.extend(seq3)
    files3 = [path.join(folder,  ii) for ii in seq]
    return files3

def readfiles(folder, bgfile, prefix):
    mk_subfolders(folder)
    bg = np.loadtxt(bgfile, skiprows=40)
    bg_q, bg_I, _,_ = bg.T
    filenames = find_n_order_files_PtSn(folder)
    nfiles = len(filenames)
    intensity_image = np.zeros((bg_q.size,nfiles),dtype=float)
    error_image = np.zeros((bg_q.size,nfiles),dtype=float)
    for ii in range(nfiles):
        spectrum = np.loadtxt(filenames[ii], skiprows=34)
        spec_q, spec_I, spec_dI, _ = spectrum.T
        if (spec_q != bg_q).any():
            raise ValueError("Issue!  domain mismatch %s" % filenames[ii])
        intensity_image[:,ii] = spec_I
        error_image[:,ii] = spec_dI
    #waterfall_plot(diff_image, bg, loc)
    return intensity_image, error_image, bg, filenames

In [5]:
intensity_image1, error_image1, bg1, files1 = readfiles(folder1, bgfile1, 'PtSn1')
intensity_image2, error_image2, bg2, files2 = readfiles(folder2, bgfile2, 'PtSn2')

Functions for individual fits

In [6]:
#def gauss_a(x, a, x0, sigma):
#    return a * np.exp(-0.5*((x - x0)/sigma)**2)

def gauss(x, I, x0, sigma):
    return I * (2*np.pi*sigma**2)**-0.5 * np.exp(-0.5*((x - x0)/sigma)**2)

def manypeaks(x, *params):
    # params pattern:  a0, x0, s0, a1, x1, s1...
    params = np.array(params)
    npeaks = params.size/3
    y = np.zeros(x.shape,dtype=float)
    for ii in range(npeaks):
        Ii,xi,si = params[3*ii:3*ii+3]
        y += gauss(x,Ii,xi,si)
    return y



def fcc2(qq, I1, x1, s1, I2ratio, s2ratio):
    # Suitable for pure Pt, Pt3Sn
    return manypeaks(qq, I1, x1, s1, I1*I2ratio, x1*((4./3)**0.5), s1*s2ratio)

def bcc2(qq, I1, x1, s1, I2, s2):
    # Suitable for PtSn
    return manypeaks(qq, I1, x1, s1, I2, x1*(0.9495**-1), s2)

def mix_fcc2_bcc2(qq,*pars):
    pars = list(pars)
    fccpars = pars[:5]
    bccpars = pars[5:]
    return fcc2(qq,*fccpars) + bcc2(qq,*bccpars)

def plot_one_guess(spec_q, diff, guess, loc, file, nn):
    fig, ax = plt.subplots()
    ax.plot(spec_q, diff, c='k', lw=1)
    ax.plot(spec_q, guess, c='k', lw=1.5)
    ax.set_title('%s guess' % path.split(file)[1])
    fig.savefig(loc + '_%i_guess.pdf' % nn)
    plt.close(fig)
    
def waterfall_plot(diff_image, bg, loc):
    nq,nfiles = diff_image.shape
    qq,_,_,_ = bg.T
    viridis = plt.get_cmap('viridis')
    cNorm = colors.Normalize(vmin=0, vmax=nfiles)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=viridis)
    fig, ax = plt.subplots()
    ax.plot([qq[0],qq[-1]], [0,0], color='k',lw=1.5)
    for ii in range(nfiles):
        colorVal = scalarMap.to_rgba(ii)
        ax.plot(qq, diff_image[:,ii], color=colorVal, lw=0.5)
    features = initial_features(bg)
    ax.plot(qq[~features],-np.ones((~features).sum()),ls='none',marker=',',c='r')
    ax.set_title('Waterfall; background regions')
    fig.savefig(loc + '_waterfall.pdf')
    #plt.close(fig)

def initial_features(bg):
    qq,_,_,_ = bg.T
    features = ~(((qq > 3.5) & (qq < 4.3)) | ((qq > 2.2) & (qq < 2.5)) | (qq > 4.5))
    return features
    
def array_plot(outputs, loc, bound_index_1, bound_index_2, nfiles):
    npars,nfiles=outputs.shape
    if npars==10:
        fig, ax = plt.subplots(3,4)
    if npars==5:
        fig, ax = plt.subplots(3,2)
    ax[0,0].plot(np.arange(nfiles), outputs[0,:])
    ax[1,0].plot(np.arange(nfiles), outputs[1,:])
    ax[2,0].plot(np.arange(nfiles), outputs[2,:])
    ax[0,1].plot(np.arange(nfiles), outputs[0,:]*outputs[3,:])
    ax[1,1].plot(np.arange(nfiles), outputs[1,:]*((4./3)**0.5))
    ax[2,1].plot(np.arange(nfiles), outputs[2,:]*outputs[4,:])
    if npars==10:
        ax[0,2].plot(np.arange(nfiles), outputs[5,:])
        ax[1,2].plot(np.arange(nfiles), outputs[6,:])
        ax[2,2].plot(np.arange(nfiles), outputs[7,:])
        ax[0,3].plot(np.arange(nfiles), outputs[8,:])
        ax[1,3].plot(np.arange(nfiles), outputs[6,:]*(0.9495**-1))
        ax[2,3].plot(np.arange(nfiles), outputs[9,:])
    for ii in range(12):
        axii = ax.flatten()[ii]
        lims = axii.axis()
        axii.plot([bound_index_1, bound_index_1], [lims[2]*1.001, lims[3]*0.999], c='r', ls='--')
        axii.plot([bound_index_2, bound_index_2], [lims[2]*1.001, lims[3]*0.999], c='r', ls='--')
    fig.suptitle('Parameter evolution')
    ax[2,1].set_xlabel('Index')
    ax[0, 0].set_ylabel('Integrated intensity')
    ax[1, 0].set_ylabel('$q$ centroid')
    ax[2, 0].set_ylabel('$q$ width ($\sigma$)')
    ax[0, 0].set_title('fcc peak 1')
    ax[0, 1].set_title('fcc peak 2')
    ax[0, 2].set_title('bcc peak 1')
    ax[0, 3].set_title('bcc peak 2')
    fig.savefig(loc + '_evolution_a.pdf')
    #plt.close(fig)

    
    
def bg_subtract_fit_individual(function, intensity_image, filenames, folder, bg, prefix, sequencestarters, initialvalues, bounds, demonstrative=False):
    loc = path.join(folder, 'addtnl', prefix)
    bg_q, bg_I, _,_ = bg.T
    nfiles = len(filenames)
    features = initial_features(bg)
    nofit = np.zeros(nfiles,dtype=bool)
    loc = path.join(folder, 'addtnl', prefix)
    p0 = initialvalues[0]
    outputs = np.zeros((len(initialvalues[0]),nfiles),dtype=float)
    diff_image = np.zeros((bg_q.size,nfiles),dtype=float)
    for ii in range(nfiles):
        spec_I = intensity_image[:,ii]
        mult = ((spec_I/bg_I)[~features]).mean()
        diff = spec_I - mult*bg_I
        diff_image[:,ii] = diff
        if filenames[ii] in sequencestarters:
            nn = sequencestarters.index(filenames[ii])
            guess = function(bg_q, *initialvalues[nn])
            plot_one_guess(bg_q, diff, guess, loc, filenames[ii], nn)
        try:
            popt, pcov = curve_fit(function, bg_q, diff, p0=p0, bounds=bounds)
        except RuntimeError:
            print '''Issue!  fitting failure %s''' % filenames[ii]
            nofit[ii] = True
            if np.isnan(diff).any():
                print '''There are NaNs in this file's data.'''
            if (diff == 0).any():
                print '''There are zero-valued entries in this file's data.'''
            figz, axz = plt.subplots()
            axz.plot(bg_q, diff)
            axz.plot(bg_q, function(bg_q, *p0), c='k', lw=1)
            axz.set_title('This spectrum has fitting issues')
        except ValueError:
            import pdb; pdb.set_trace()
        p0 = popt
        outputs[:,ii] = popt
        if demonstrative:
            plot_one_fit(bg_q, diff, function(bg_q, *p0), folder, filenames[ii])
    #
    waterfall_plot(diff_image, bg, loc)
    bound_index_1 = filenames.index(sequencestarters[1])
    bound_index_2 = filenames.index(sequencestarters[2])
    #array_plot_3(outputs, loc, bound_index_1, bound_index_2, nfiles)
    array_plot(outputs, loc, bound_index_1, bound_index_2, nfiles)
    '''
    fname = path.join(folder, 'addtnl', prefix) + '_evolution.csv'
    headertxt = 'index, peak height 1, q centroid 1, sigma width 1, integrated intensity 1, FWHM 1, '\
                + 'peak height 2, q centroid 2, sigma width 2, integrated intensity 2, FWHM 2, '\
                + 'peak height 3, q centroid 3, sigma width 3, integrated intensity 3, FWHM 3, '\
                + 'peak height 4, q centroid 4, sigma width 4, integrated intensity 4, FWHM 4'
    np.savetxt(fname, outdata.T, fmt='%.18f', delimiter=',', newline='\n', header=headertxt)
    '''
    #
    fig6, ax6 = plt.subplots()
    ax6.imshow(diff_image.T)
    ax6.set_xlabel('q index')
    ax6.set_ylabel('t index')
    ax6.set_title('Background subtracted spectrum evolution')
    fname = (path.join(folder, 'addtnl', prefix) + '_diffimage.pdf')
    fig6.savefig(fname)
    plt.close(fig6)
    #
    return outputs, intensity_image, diff_image, nofit

In [8]:
outputs1, intensity_image1, diff_image1, nofit1 = bg_subtract_fit_individual(mix_fcc2_bcc2,intensity_image1, files1, folder1, bg1, 'PtSn1', sequencestarters1, initialvalues1, bounds1)
outputs2, intensity_image2, diff_image2, nofit2 = bg_subtract_fit_individual(mix_fcc2_bcc2,intensity_image2, files2, folder2, bg2, 'PtSn2', sequencestarters2, initialvalues2, bounds2)

Issue!  fitting failure /Users/Amanda/Desktop/PtSn/PtSn2/_PtSn2_scan1_0037_C.dat


Fast records of low-constraint fits for Liheng.

In [ ]:
bg_q1, _,_,_ = bg1.T
fit_image1 = np.zeros(intensity_image1.shape, dtype=float)
for ii in range(intensity_image1.shape[1]):
    fit_image1[:,ii] = manypeaks(bg_q1, *outputs1[:,ii])
#bg_q2, _,_,_ = bg2.T
#fit_image2 = np.zeros(intensity_image2.shape, dtype=float)
#for ii in range(intensity_image2.shape[1]):
#    fit_image2[:,ii] = manypeaks(bg_q2, *outputs2[:,ii])
#fname = path.join(folder, 'addtnl', prefix) + '_evolution.csv'
headertxt = 'integrated intensity 1, q centroid 1, sigma width 1, '\
            + 'integrated intensity 2, q centroid 2, sigma width 2, '\
            + 'integrated intensity 3, q centroid 3, sigma width 3, '\
            + 'integrated intensity 4, q centroid 4, sigma width 4'
np.savetxt(fname, outputs.T, fmt='%.18f', delimiter=',', newline='\n', header=headertxt)



Functions for context-aware background subtraction

In [ ]:
def local_error_estimate(image):
    error_estimate = np.zeros(image.shape, dtype=float)
    error_estimate[1:-1,1:-1] = (image[1:-1,1:-1] - image[1:-1,2:])**2 \
                                + (image[1:-1,1:-1] - image[1:-1,:-2])**2 \
                                + (image[1:-1,1:-1] - image[2:,1:-1])**2 \
                                + (image[1:-1,1:-1] - image[:-2,1:-1])**2
    error_estimate[0,:] = error_estimate[1,:]
    error_estimate[-1,:] = error_estimate[-2,:]
    error_estimate[:,0] = error_estimate[:,1]
    error_estimate[:,-1] = error_estimate[:,-2]
    error_estimate = (error_estimate / 8.) ** 0.5
    return error_estimate

def grow(arr, nn):
    arr = arr.astype('bool')
    for ii in range(nn):
        arr[1:,:] = arr[1:,:] | arr[:-1,:]
        arr[:-1,:] = arr[1:,:] | arr[:-1,:]
        arr[:,1:] = arr[:,1:] | arr[:,:-1]
        arr[:,:-1] = arr[:,1:] | arr[:,:-1]
    return arr

def twodeebg(intensity, bg_I, legit):
    nq, nfiles = intensity.shape
    blank = np.ones((nq, nfiles))
    mult = np.ones(nfiles) #*blank
    if len(bg_I.shape) == 1:
        bg_I = bg_I.reshape((nq,1))*blank
    mult[legit.any(axis=0)] = ((intensity / bg_I * legit).sum(axis=0)/legit.sum(axis=0,dtype=float)).reshape((nfiles,))[legit.any(axis=0)]  # potential problem line
    if np.any(~legit.any(axis=0)):
        print "Some spectra/timeframes are entirely deselected.  Consider re-tuning your cutoff values."
    mult = mult.reshape((1,nfiles))
    #diff = intensity - mult*bg_I
    diff = intensity/mult - bg_I
    return mult, diff

def plotarray(nplots, arrange_vertical=True):
    nplots = int(nplots)
    if arrange_vertical:
        nrows = int(np.ceil(nplots**0.5))
        ncols = int(np.ceil(nplots/nrows))
    else:
        ncols = int(np.ceil(nplots**0.5))
        nrows = int(np.ceil(nplots/ncols))
    fig, ax = plt.subplots(nrows,ncols)
    flatax = ax.ravel()
    for ii in range(ax.size):
        if ii > (nplots - 1):
            flatax[ii].axis('off')
    return fig, ax

def tree_bg(intensity_image, diff_image, bg, cutoff, tree_depth, max_iter=15, demonstrative=False, loc=None):
    '''"Demonstrative" is like "verbose", but with plots instead of words.'''
    nq, nfiles = intensity_image.shape
    blank = np.ones((nq, nfiles), dtype=float)
    qq = bg[:, 0]
    bg_I = bg[:, 1]
    index = np.arange(nfiles)
    X1 = (qq.reshape(nq,1)*blank).flatten()
    X2 = (index.reshape(1,nfiles)*blank).flatten()
    X = np.array(zip(X1,X2))
    clf = tree.DecisionTreeClassifier(max_depth=tree_depth)
    oldpredictions = []
    for ii in range(max_iter):
        smooth_diff = medfilt(diff_image, 3)
        error_estimate = local_error_estimate(diff_image)
        smooth_err = medfilt(error_estimate, 3)
        frame4 = (np.fabs(smooth_diff) / cutoff > smooth_err)
        frame5 = medfilt(frame4, 3)
        frame6 = grow(frame5, 2)
        Y = frame6.flatten()
        clfsol = clf.fit(X, Y)
        prediction = (clfsol.predict(X)).reshape(nq, nfiles)
        metrics = []
        for jj in oldpredictions:
            metric = (jj != prediction).sum()
            metrics.append(metric)
        if ii != 0:
            metricbest = min(metrics)
            indexbest = metrics.index(metricbest)
            if metricbest == 0:
                if indexbest == (len(metrics) - 1):
                    print "Converged, iteration %i" % ii
                    suffix = ""
                else:
                    cyclelength = (len(metrics) - indexbest)
                    print "Cyclical convergence, iteration %i, cycle length %i" % (ii, cyclelength)
                    suffix = ", cycle length %i" % cyclelength
                    for kk in range(1, cyclelength):
                        prediction = (prediction | oldpredictions[-kk])
                        oldpredictions.append(prediction)
                break
        oldpredictions.append(prediction)
        mult, diff_image = twodeebg(intensity_image, bg_I, ~prediction)
    if demonstrative:
        nplots = len(oldpredictions)
        fig, ax = plotarray(nplots)
        flatax = ax.ravel()
        for ii in range(nplots):
            axii = flatax[ii]
            if ii < nplots:
                axii.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off',
                                labelleft='off')
                axii.imshow(oldpredictions[ii].T, cmap='binary')
                if ii != (nplots-1):
                    axii.set_title(ii)
                else:
                    axii.set_title("Final")
        fig.suptitle("Evolution of decision tree selection" + suffix)
        fig2, ax2 = plt.subplots()
        ax2.imshow(diff_image.T,cmap='viridis')
        ax2.axis('off')
        ax2.set_title("Final background-subtracted spectra")
        if loc is not None:
            fname = (loc + '_dtree_bgselection_evolution.pdf')
            fig.savefig(fname)
            fname2 = (loc + '_diffimage_final.pdf')
            fig2.savefig(fname2)
        #fig.set_figheight(8)
        #fig.set_figwidth(12)
        #return prediction, mult, diff_image, fig, fig2
    return prediction, mult, diff_image

In [ ]:
feature_map1, mult1, final_diff_image1 = tree_bg(intensity_image1, diff_image1, bg1, 1.5, 5, demonstrative=True, loc=loc1)
feature_map2, mult2, final_diff_image2 = tree_bg(intensity_image2, diff_image2, bg2, 1.5, 5, demonstrative=True, loc=loc2)

In [ ]:
def pseudovoigt_a(x, aG, aL, x0, sigma):
    x_ = (x - x0)/sigma
    return aG*np.exp(-0.5*(x_**2)) + aL*(1 + (x_**2))**-1

def pseudovoigt_I(x, I, eta, x0, sigma):
    x_ = (x - x0)/sigma
    cG = (2*np.pi*sigma**2)**-0.5
    cL = (np.pi*sigma)**-1
    return I*eta*cG*np.exp(-0.5*(x_**2)) + I*(1-eta)*cL*(1 + (x_**2))**-1


def PtSn1_bounds(nfiles):
    
    # a1, x1, deltax1, s1, a1factor, s1factor, a2, x2, deltax2, s2, a2factor, s2factor
    bounds_lo = np.zeros((2*nfiles+10,), dtype=float)
    bounds_hi = np.ones((2*nfiles+10,), dtype=float)*np.inf
    #bounds_lo[0:nfiles] = #a1
    bounds_lo[nfiles] = 2.69 #x1
    bounds_lo[nfiles+1] = -0.002 #deltax1
    bounds_lo[nfiles+2] = 0.025 #s1
    bounds_lo[nfiles+3] = 0.05 #a1factor_0
    bounds_lo[nfiles+4] = 0.5 #s1factor
    #bounds_lo[nfiles+5:2*nfiles+5] = #a2
    bounds_lo[2*nfiles+5] = 2.80 #x2
    bounds_lo[2*nfiles+6] = -0.002 #deltax2
    bounds_lo[2*nfiles+7] = 0.025 #s2
    bounds_lo[2*nfiles+8] = 0.2 #a2factor_0
    bounds_lo[2*nfiles+9] = 0.5 #s2factor_0
    
    bounds_hi[0:nfiles] = 12 #a1
    bounds_hi[nfiles] = 2.79 #x1
    bounds_hi[nfiles+1] = 0.002 #deltax1
    bounds_hi[nfiles+2] = 0.08 #s1
    bounds_hi[nfiles+3] = 1. #a1factor_0
    bounds_hi[nfiles+4] = 2. #s1factor
    bounds_hi[nfiles+5:2*nfiles+5] = 6.5 #a2
    bounds_hi[2*nfiles+5] = 2.90 #x2
    bounds_hi[2*nfiles+6] = 0.002 #deltax2
    bounds_hi[2*nfiles+7] = 0.08 #s2
    bounds_hi[2*nfiles+8] = 1. #a2factor_0
    bounds_hi[2*nfiles+9] = 2. #s2factor_0
    return [bounds_lo, bounds_hi]

def gauss(x, a, x0, sigma):
    #return (2*np.pi*sigma**2)**-0.5 * np.exp(-0.5*((x - x0)/sigma)**2)
    return a * np.exp(-0.5*((x - x0)/sigma)**2)

def multipeaks1(x, *params):
    '''Group of peaks with constant amplitude ratios, constant location ratios, constant width ratios'''
    # params pattern:  a0, x0, s0, a1factor, x1factor, s1factor, a2factor, x2factor, s2factor...
    params = np.array(params)
    npeaks = params.size/3
    y = np.zeros(x.shape,dtype=float)
    a0, x0, s0 = params[0:3]
    for ii in range(npeaks):
        afactori,xfactori,sfactori = params[3*ii:3*ii+3]
        ai = afactori*a0
        xi = xfactori*x0
        si = sfactori*s0
        y += gauss(x,ai,xi,si)
    return y

def pseudovoigt(x, aG, aL, x0, sigma):
    x_ = (x - x0)/sigma
    return aG*np.exp(-0.5*(x_**2)) + aL*(1 + (x_**2))**-1

def interpret(opt):
    aG, aL, x0, sigma = opt
    h = aG + aL
    I = aG*sigma*(2*np.pi)**0.5 + aL*np.pi*sigma
    #return h, I, x0, sigma
    return I, x0, sigma


def multipeaks2(x, *params):
    '''Group of pseudovoigt peaks with constant integrated intesity ratios, constant location ratios, constant width ratios'''
    # params pattern:  a0, x0, s0, a1factor, x1factor, s1factor, a2factor, x2factor, s2factor...
    params = np.array(params)
    npeaks = params.size/3
    y = np.zeros(x.shape,dtype=float)
    a0, x0, s0 = params[0:3]
    for ii in range(npeaks):
        afactori,xfactori,sfactori = params[3*ii:3*ii+3]
        ai = afactori*a0
        xi = xfactori*x0
        si = sfactori*s0
        y += gauss(x,ai,xi,si)
    return y

def linear2(ii, b, a):
    return ii*a + b

def Pt_phase1(qq, a1, x1, s1, afactor, sfactor):
    return multipeaks(qq, a1, x1, s1, afactor, (4./3)**0.5, sfactor)

def PtSn_q2q3(qq, a1, x1, s1, afactor, sfactor):
    return multipeaks(qq, a1, x1, s1, afactor, 0.9495**-1, sfactor)

def PtSn1_full(X, *params):
    # a1, x1, deltax1, s1, a1factor, s1factor, a2, x2, deltax2, s2, a2factor, s2factor
    #nq, nfiles = diff_image.shape
    qq, ii = X
    params = np.array(params)
    nfiles = (params.size - 10)/2
    nq = qq.size/nfiles
    a1 = params[:nfiles]
    x1 = params[nfiles]
    deltax1 = params[nfiles+1]
    s1 = params[nfiles+2]
    a1factor = params[nfiles+3]
    s1factor = params[nfiles+4]
    a2 = params[nfiles+5:2*nfiles+5]
    x2 = params[nfiles+5]
    deltax2 = params[nfiles+6]
    s2 = params[nfiles+7]
    a2factor_0 = params[nfiles+8]
    s2factor_0 = params[nfiles+9]
    A1 = (a1[np.newaxis,:]*np.ones((nq,nfiles),dtype=float)).flatten()
    X1 = linear2(ii, x1, deltax1)
    S1 = s1*np.ones(X1.shape)
    A2 = (a2[np.newaxis,:]*np.ones((nq,nfiles),dtype=float)).flatten()
    X2 = linear2(ii, x2, deltax2)
    S2 = s2*np.ones(X2.shape)
    return Pt_phase1(qq, A1, X1, S1, a1factor, s1factor) + PtSn_q2q3(qq, A2, X2, S2, a2factor_0, s2factor_0)

def duo_guess(iis, a1s, x1s, s1s, a2s, x2s, s2s):
    (x1, x1slope), _ = curve_fit(linear2, iis, x1s, p0=[x1s.mean(), 0])
    s1 = s1s.mean()
    s1factor = s2s.mean()/s1
    a1factor = a2s.mean()/a1s.mean()
    x1factor = x2s.mean()/x1
    a1 = (a1s + a2s)/(1 + a1factor)
    return a1, x1, x1slope, s1, a1factor, x1factor, s1factor

def guess_from_outputs(outputs):
    nfeatures, nfiles = outputs.shape
    # unpacking outputs
    a1_0 = outputs[0,:]
    x1_0 = outputs[1,:]
    s1_0 = outputs[2,:]
    a2_0 = outputs[3,:]
    x2_0 = outputs[4,:]
    s2_0 = outputs[5,:]
    a3_0 = outputs[6,:]
    x3_0 = outputs[7,:]
    s3_0 = outputs[8,:]
    a4_0 = outputs[9,:]
    x4_0 = outputs[10,:]
    s4_0 = outputs[11,:]
    ii_0 = np.arange(a1_0.size, dtype=float)
    # initial param guesses
    # a1, x1, s1, a2, x2, s2, a3, x3, s3, a4, x4, s4 in outputs
    # vs
    # a1, x1, s1, a1factor, x1factor, a2, x2, s2, a2factor, x2 factor for full fit
    a1, x1, deltax1, s1, a1factor_0, x1factor_0, s1factor_0 = duo_guess(ii_0, a1_0, x1_0, s1_0, a4_0, x4_0, s4_0)
    a2, x2, deltax2, s2, a2factor_0, x2factor_0, s2factor_0 = duo_guess(ii_0, a2_0, x2_0, s2_0, a3_0, x3_0, s3_0)
    a_conglomerate = np.zeros((2*nfiles,))
    a_conglomerate[:nfiles] = a1_0
    a_conglomerate[nfiles:] = a2_0
    guesses = [a1_0, x1, deltax1, s1, a1factor_0, x1factor_0, s1factor_0]
    guesses.extend([a2_0, x2, deltax2, s2, a2factor_0, x2factor_0, s2factor_0])
    #return a1, x1, deltax1, s1, a1factor, s1factor, a2, x2, deltax2, s2, a2factor, s2factor
    print "guesses size", len(guesses), "should be 14"
    return guesses


def pack_list_of_lists_to_array(lol):
    '''Converts [[a,b],[c],d] to array([a,b,c,d]).'''
    n_elements = 0
    for ii in range(len(lol)):
        try: # type(lol[ii]) in [list, np.ndarray, tuple]
            n_elements += len(lol[ii])
        except TypeError:
            n_elements += 1
    lolarray = np.zeros((n_elements,),dtype=float)
    element = 0
    for ii in range(len(lol)):
        try: # type(lol[ii]) in [list, np.ndarray, tuple]
            lolarray[element:element+len(lol[ii])] = lol[ii]
            element += len(lol[ii])
        except TypeError:
            lolarray[element] = lol[ii]
            element += 1
    return lolarray

def PtSn1paramfromguess(guess):
    a1, x1, deltax1, s1, a1factor_0, x1factor_0, s1factor_0 = guess[:7]
    a2, x2, deltax2, s2, a2factor_0, x2factor_0, s2factor_0 = guess[7:]
    print "x1factor guess is %f; expected %f" % (x1factor_0, (4./3)**0.5)
    print "x2factor guess is %f; expected %f" % (x2factor_0, 0.9495**-1)
    guess.pop(13)
    guess.pop(6)
    params = pack_list_of_lists_to_array(guess)
    return params

def inbounds(p0, bounds):
    for ii in range(len(p0)):
        if p0[ii] < bounds[0][ii]:
            p0[ii] = bounds[0][ii]
            print "Parameter at index %i was raised because out of bounds" %ii
        if p0[ii] > bounds[1][ii]:
            p0[ii] = bounds[1][ii]
            print "Parameter at index %i was lowered because out of bounds" %ii
        if bounds[0][ii] > bounds[1][ii]:
            raise ValueError("Lower bound must be <= upper bound.  Exception at index %i"%ii)
    return p0

def fit_collective(diff_image, bg, outputs, funcname, nofit, demonstrative=False):
    diff_image = diff_image[:,~nofit]
    outputs = outputs[:,~nofit]
    nq, nfiles = diff_image.shape
    blank = np.ones((nq, nfiles), dtype=float)
    qq,_,_,_ = bg.T
    #qq,bg_I,_,_ = bg.T
    index = np.arange(nfiles)
    qq_big = qq.reshape(nq,1)*blank
    X1 = qq_big.flatten()
    index_big = index.reshape(1,nfiles)*blank
    X2 = index_big.flatten()
    X = np.array((X1,X2))
    Y = diff_image.flatten()
    #a1, deltaa1, x1, deltax1, s1, deltas1, a1factor, x1factor, a2, deltaa2, x2, deltax2, s2, deltas2, a2factor, x2factor = guess_from_outputs(outputs)
    #a1, x1, deltax1, s1, deltas1, a1factor, x1factor, a2, x2, deltax2, s2, deltas2, a2factor, x2factor = guess_from_outputs(outputs)
    if funcname == 'PtSn1_full':
        # X, a1, x1, deltax1, s1, deltas1, a1factor, _, a2, x2, deltax2, s2, deltas2, a2factor, _
        # only a1 and a2 are arrays size 180
        p0 = PtSn1paramfromguess(guess_from_outputs(outputs))
        bounds = PtSn1_bounds(nfiles)
        p0 = inbounds(p0,bounds)
        popt, pcov = curve_fit(PtSn1_full, X, Y, p0=p0, bounds=bounds)
        fit_result = PtSn1_full(X, *popt).reshape(nq,nfiles)
    elif funcname == 'PtSn2_full':
        # X, a1, x1, deltax1, s1, deltas1, a1factor, _, a2, x2, deltax2, s2, deltas2, a2factor, _
        # only a1 and a2 are arrays size 180
        p0 = PtSn2paramfromguess(guess_from_outputs(outputs))
        p0 = inbounds(p0,bounds)
        popt, pcov = curve_fit(PtSn2_full, X, Y, p0=p0)
        fit_result = PtSn2_full(X, *popt).reshape(nq,nfiles)
    else:
        raise ValueError("Function name not recognized.")
    if demonstrative:
        fig, ax = plt.subplots()
        ax.imshow(fit_result.T)
    return popt, pcov, fit_result


In [ ]:
'''
popt1, pcov1, fit_result1 = fit_collective(final_diff_image1, bg1, outputs1, 'PtSn1_full', nofit1, True)
'''

In [ ]:
'''
popt2, pcov2, fit_result2 = fit_collective(final_diff_image2, bg2, outputs2, 'PtSn2_full', nofit2, True)
'''

In [ ]:
print len(files1)

In [ ]:
def pack_list_of_lists_to_array(lol):
    '''Converts [[a,b],[c],d] to array([a,b,c,d]).
    
    Sequence (array, list, tuple) and singleton (number) list elements allowed.
    
    Array elements are expected to come pre-flattened.  Result is assumed to be float type.
    '''
    n_elements = 0
    for ii in range(len(lol)):
        try: # type(lol[ii]) in [list, np.ndarray, tuple]
            n_elements += len(lol[ii])
        except TypeError:
            n_elements += 1
    lolarray = np.zeros((n_elements,),dtype=float)
    element = 0
    for ii in range(len(lol)):
        try: # type(lol[ii]) in [list, np.ndarray, tuple]
            lolarray[element:element+len(lol[ii])] = lol[ii]
            element += len(lol[ii])
        except TypeError:
            lolarray[element] = lol[ii]
            element += 1
    return lolarray

def pattern_unpack(pattern, packarray):
    '''Inverts the work of pack_list_of_lists_to_array.'''
    unpacked = []
    element = 0
    n_elements = packarray.size
    for ii in range(len(pattern)):
        try:
            unpacked.append(packarray[element:element+len(pattern[ii])])
            element += len(pattern[ii])
        except TypeError:
            unpacked.append(packarray[element])
            element += 1
        except IndexError:
            print "The pattern seems to have more elements than the array to be unpacked."
    if element < (n_elements-1):
        raise IndexError("The pattern seems to have fewer elements than the array to be unpacked.")
    return unpacked
    

print pack_list_of_lists_to_array([[1,2],[3],4])

In [ ]:
def Pt_lowconstraint(qq, a1, x1, s1, a2, s2):
    return manypeaks(qq, a1, x1, s1, a2, x1*(4./3)**0.5, s2)

def PtSn_lowconstraint(qq, a1, x1, s1, a2, s2):
    return manypeaks(qq, a1, x1, s1, a2, x1*0.9495**-1, s2)

def hybrid_Pt_PtSn_lowconstraint(qq, *pars):
    pars1 = list(pars)[:5]
    pars2 = list(pars)[5:]
    return Pt_lowconstraint(qq, *pars1) + Pt_lowconstraint(qq, *pars2)
